In [1]:
import torch
from tqdm import tqdm_notebook as tqdm

from transformers import BertTokenizer, BertModel, BertForMaskedLM

I0702 05:20:18.803425 140581000734528 file_utils.py:39] PyTorch version 1.1.0 available.


In [2]:
import pandas as pd
import numpy as np

In [3]:
from torch.nn.utils.rnn import pad_sequence 
import torchvision
import torch.nn as nn

In [4]:
from transformers.optimization import AdamW
from transformers.optimization import AdamW, get_linear_schedule_with_warmup

In [5]:
validation_data = pd.read_json("FGC_release_1.7.13/FGC_release_all_dev.json")
training_data = pd.read_json("FGC_release_1.7.13/FGC_release_all_train.json")
test_data = pd.read_json("FGC_release_1.7.13/FGC_release_all_test.json")

In [6]:
device = torch.device("cuda")

## Data Preprocessing

In [7]:
def datapreprocessing(data, is_dev=False):
    
    # Save all the questions, potential supporting evidence and indices in three lists
    textQ_to_be_tokenized = []
    textA_to_be_tokenized = []
    sp_index = []
    
    for dictionary in data['QUESTIONS']:
        for element in dictionary:
            textQ_to_be_tokenized.append(element['QTEXT_CN'])
            sp_index.append(element['SHINT_'])
    for dictionary in data['SENTS']:
        current_text_sentence = []
        for element in dictionary:
            current_text_sentence.append(element['text'])
        textA_to_be_tokenized.append(current_text_sentence)
    
    QandA_label = pd.DataFrame({'Question': textQ_to_be_tokenized,
                                'Sentence_List': textA_to_be_tokenized,
                                'SE_Index': sp_index,
                                'Label': sp_index})
    
    QandA_label['Length'] = QandA_label['Sentence_List'].apply(lambda x: len(x))
    QandA_label['SE_Index'] = QandA_label['SE_Index'].apply(lambda x: [0])
    QandA_label['SE_Index'] = QandA_label['SE_Index'] * QandA_label['Length']
    QandA_label['SE_Index'] = list(zip(QandA_label['SE_Index'], QandA_label['Label']))

    # Extract label index
    for row in QandA_label['SE_Index']:
        for index in row[1]:
            row[0][index] = 1
        
    indexed = [i[0] for i in list(QandA_label['SE_Index'])]
    QandA_label['Label'] = indexed

    Q_and_Sentence_all_Comb = pd.DataFrame({'Question':np.repeat(QandA_label['Question'].values, QandA_label['Sentence_List'].str.len()),
                        'Sentence':np.concatenate(QandA_label['Sentence_List'].values)})
    Q_and_Sentence_all_Comb['Label'] = QandA_label['Label'].sum()
    
    tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
            
    # Put all question and sentence combination into a list 
    All_instances = []
    for i in range(len(QandA_label)):
        for sentence in QandA_label['Sentence_List'][i]:
            question_token = tokenizer.tokenize(QandA_label['Question'][i])
            sentence_token = tokenizer.tokenize(sentence)
            instance = ['[CLS]'] + question_token + ['[SEP]'] + sentence_token + ['[SEP]']
            if len(instance) > 512:
                instance = instance[:512]
            All_instances.append(instance)
            
    # Convert ids to segment_ids
    segment_ids = []
    for token in All_instances:
        length_of_zeros = token.index('[SEP]') - token.index('[CLS]') + 1
        length_of_ones = len(token) - length_of_zeros
        zeros_and_ones = [0] * length_of_zeros + [1] * length_of_ones
        segment_ids.append(zeros_and_ones)
        
    ids = []
    for token in All_instances:
        ids.append(tokenizer.convert_tokens_to_ids(token))
        
    mask_ids = []
    for token in All_instances:
        mask_ids.append([1] * len(token))
        
    labels = list(Q_and_Sentence_all_Comb['Label'])
    labels = [[i] for i in labels]
    
    return All_instances, ids, segment_ids, mask_ids, labels

In [8]:
dev_instances, dev_ids, dev_seg_ids, dev_mask_ids, dev_labels = datapreprocessing(validation_data, True)
train_instances, train_ids, train_seg_ids, train_mask_ids, train_labels = datapreprocessing(training_data)

I0702 05:20:22.738082 140581000734528 tokenization_utils.py:375] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-chinese-vocab.txt from cache at /root/.cache/torch/transformers/8a0c070123c1f794c42a29c6904beb7c1b8715741e235bee04aca2c7636fc83f.9b42061518a39ca00b8b52059fd2bede8daa613f8a8671500e518a8c29de8c00
I0702 05:20:27.630849 140581000734528 tokenization_utils.py:375] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-chinese-vocab.txt from cache at /root/.cache/torch/transformers/8a0c070123c1f794c42a29c6904beb7c1b8715741e235bee04aca2c7636fc83f.9b42061518a39ca00b8b52059fd2bede8daa613f8a8671500e518a8c29de8c00


## Loading Data

In [9]:
from torch.utils.data import Dataset

In [10]:
class SentenceDataset(Dataset):
    
    def __init__(self, ids, segment_ids, mask_ids, labels):
        self.instances = []
        for ids_i, segment_ids_i, mask_ids, label in zip(ids, segment_ids, mask_ids, labels):
            self.instances.append({"ids": ids_i, "segment_ids": segment_ids_i, 
                                   "mask_ids": mask_ids, "labels": label})  
    def __len__(self):
        return len(self.instances)

    def __getitem__(self, idx):
        sample = self.instances[idx]

        return sample

In [11]:
train_dataset = SentenceDataset(train_ids, train_seg_ids, train_mask_ids, train_labels)

In [12]:
dev_dataset = SentenceDataset(dev_ids, dev_seg_ids, dev_mask_ids, dev_labels)

In [13]:
from torch.utils.data import DataLoader

In [14]:
def collate(batch):
    padded_ids = pad_sequence([torch.tensor(instance['ids']) for instance in batch], batch_first=True)
    padded_ids = padded_ids.to(device)
    
    padded_segment_ids = pad_sequence([torch.tensor(instance['segment_ids']) for instance in batch], batch_first=True)
    padded_segment_ids = padded_segment_ids.to(device)
    
    padded_mask_ids = pad_sequence([torch.tensor(instance['mask_ids']) for instance in batch], batch_first=True)
    padded_mask_ids = padded_mask_ids.to(device)
    
    labels = torch.stack([torch.tensor(instance['labels']) for instance in batch])
    labels = labels.to(device)
    return {'ids': padded_ids, 'segment_ids': padded_segment_ids, 'mask_ids': padded_mask_ids, 'labels': labels}

In [15]:
dataloader_train = DataLoader(train_dataset, batch_size=8, shuffle = True, collate_fn = collate)
#dataloader_dev = DataLoader(dev_dataset, collate_fn = collate)

In [16]:
dev_batches = []
len_array = np.cumsum(np.array(validation_data['SENTS'].apply(lambda x: len(x))))

dictionary_lists = []
dev_batches = []
for i in range(len(dev_dataset.instances)):
    
    instance = dev_dataset.instances[i]
    dictionary_lists.append(instance)
    
    if i in len_array - 1:
        
        padded_ids = torch.transpose(pad_sequence([torch.tensor(instance['ids']) for instance in dictionary_lists]), 0, 1)
        padded_ids = padded_ids.to(device)
    
        padded_segment_ids = torch.transpose(pad_sequence([torch.tensor(instance['segment_ids']) for instance in dictionary_lists]), 0, 1)
        padded_segment_ids = padded_segment_ids.to(device)
    
        padded_mask_ids = torch.transpose(pad_sequence([torch.tensor(instance['mask_ids']) for instance in dictionary_lists]), 0, 1)
        padded_mask_ids = padded_mask_ids.to(device)
    
        labels = torch.stack([torch.tensor(instance['labels']) for instance in dictionary_lists])
        labels = labels.to(device)
    
        current_dev_batch = {'ids': padded_ids, 'segment_ids': padded_segment_ids, 'mask_ids': padded_mask_ids, 'labels': labels}
    
        dev_batches.append(current_dev_batch)
        dictionary_lists = []



## Creating Neural Network

In [17]:
class FGC_Network(nn.Module):

    def __init__(self):
        
        super(FGC_Network, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-chinese')
        self.linear = nn.Linear(768, 1)

    def forward(self, batch):
        # batch['ids'] = (batch_size, sent_len)
        # batch['segment_ids'] = (batch_size, sent_len)
        # batch['mask_ids'] = = (batch_size, sent_len)
        # output = (batch_size, 1)
        hidden_state, pooler_output = self.bert(batch['ids'], batch['segment_ids'], batch['mask_ids'])
        linear_output = self.linear(pooler_output)
        
        return linear_output

    def loss(self, batch):
        
        loss_fn = nn.BCEWithLogitsLoss()
        output = self.forward(batch)
        target = batch['labels'].float().to(device)
        
        return loss_fn(output, target)
    
    def _predict(self, batch):
        
        output = self.forward(batch)
        scores = torch.sigmoid(output)
        scores = scores.cpu().numpy()[:,0].tolist()
        
        return scores
    
    def predict_fgc(self, batch, threshold=0.5):
        
        scores = self._predict(batch)

        max_i = 0
        max_score = 0
        sp = []
        
        for i, score in enumerate(scores):

            if score > max_score:
                max_i = i
                max_score = score
            if score >= threshold:
                sp.append(i)

        if not sp:
            sp.append(max_i)

        return {'sp': sp, 'sp_scores': scores}

In [18]:
network = FGC_Network()
network.to(device)

I0702 05:20:47.823648 140581000734528 configuration_utils.py:152] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-chinese-config.json from cache at /root/.cache/torch/transformers/8a3b1cfe5da58286e12a0f5d7d182b8d6eca88c08e26c332ee3817548cf7e60a.f12a4f986e43d8b328f5b067a641064d67b91597567a06c7b122d1ca7dfd9741
I0702 05:20:47.828250 140581000734528 configuration_utils.py:169] Model config {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 2,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "pad_token_id": 0,
  

FGC_Network(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm(torch.Size([768]), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm(torch.Size([768]), eps=1e-12, elementwise_affine=True)
        

In [19]:
def optim(nn, num_epochs, lr):
    param_optimizer = list(nn.bert.named_parameters())
    no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]
    num_epochs = num_epochs
    num_train_optimization_steps = len(dataloader_train) * num_epochs
    optimizer = AdamW(optimizer_grouped_parameters, lr=lr)
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=100, num_training_steps=num_train_optimization_steps)
    
    return optimizer, scheduler

In [20]:
def _update_sp(metrics, sp_gold, sp_pred):
    tp, fp, fn = 0, 0, 0
        
    for p in sp_pred:
        if p in sp_gold:
            tp += 1
        else:
            fp += 1
    for g in sp_gold:
        if g not in sp_pred:
            fn += 1
            
    precision = 1.0 * tp / (tp + fp) if tp + fp > 0 else 0.0
    recall = 1.0 * tp / (tp + fn) if tp + fn > 0 else 0.0
    f1 = 2 * precision * recall / (precision + recall) if precision + recall > 0 else 0.0
    em = 1.0 if fp + fn == 0 else 0.0
    
    metrics['sp_em'] += em
    metrics['sp_f1'] += f1
    metrics['sp_prec'] += precision
    metrics['sp_recall'] += recall
    
    return precision, recall, f1

In [21]:
def eval_sp_fgc(sp_golds, sp_preds):
    
    metrics = {'sp_em': 0, 'sp_prec': 0, 'sp_recall': 0, 'sp_f1': 0}
    
    assert len(sp_golds) == len(sp_preds)
    
    for sp_gold, sp_pred in zip(sp_golds, sp_preds):
        _update_sp(metrics, sp_gold, sp_pred)
    
    N = len(sp_golds)
    for k in metrics.keys():
        metrics[k] /= N
        metrics[k] = round(metrics[k], 3)
    print(metrics)
    return metrics

In [22]:
def eval_fgc_atype(atype_golds, atype_preds):
    
    pos = 0
    neg = 0
    
    for gold, atype in zip(atype_golds, atype_preds):
        if atype == gold:
            pos += 1
        else:
            neg += 1
    return pos/len(atypes_preds)

In [23]:
def eval(network, dev_batches, current_epoch, sp_golds, avg_loss):
    
    network.eval()
    
    with torch.no_grad():
        sp_preds = []
        #atype_preds = []
        for batch in tqdm(dev_batches):
            
            out_dct = network.predict_fgc(batch)
            sp_preds.append(out_dct['sp'])
            #if 'atype' in out_dct:
                #for type_i in out_dct['atype']:
                    #assert type_i == out_dct['atype'][0]
                #atype_preds.append(type_i)
                
  
    metrics = eval_sp_fgc(sp_golds, sp_preds)
    print('epoch %d eval_recall: %.3f eval_f1: %.3f loss: %.3f' % (
            current_epoch, metrics['sp_recall'], metrics['sp_f1'], avg_loss))
        
    torch.save(network.state_dict(), "FGC_release_1.7.13/models/model_epoch{0}_eval_em:{1:.3f}_precision:{2:.3f}_recall:{3:.3f}_f1:{4:.3f}_loss:{5:.3f}.m".format(current_epoch, metrics['sp_em'], metrics['sp_prec'], metrics['sp_recall'], metrics['sp_f1'], avg_loss))

In [24]:
def train(network, num_epochs, lr):
    
    loss_fn = nn.BCEWithLogitsLoss()
    optimizer, scheduler = optim(network, num_epochs, lr)
    
    sp_golds = validation_data['QUESTIONS'].apply(lambda x: x[0]['SHINT_']).tolist()
    #atype_golds = validation_data['QUESTIONS'].apply(lambda x: x[0]['ATYPE_']).tolist()
    
    for current_epoch in range(num_epochs):
        network.train()
        running_loss = 0.0
        dr = True
        for batch in tqdm(dataloader_train):
            optimizer.zero_grad()
            current_output = network(batch)
            current_target = batch['labels'].to(dtype=torch.float, device=device)
            current_loss = loss_fn(current_output, current_target)
            if dr:
                print(current_loss)
                dr = False
            current_loss.backward()
            optimizer.step()
            scheduler.step()
            running_loss += current_loss.item()
            
        learning_rate_scalar = scheduler.get_lr()[0]
        print('lr = %f' % learning_rate_scalar)
        avg_loss = running_loss/len(dataloader_train)
        print('epoch %d train_loss: %.3f' % (current_epoch, avg_loss))
        eval(network, dev_batches, current_epoch, sp_golds, avg_loss)

In [26]:
train(network, 20, 0.00001)

tensor(0.3988, device='cuda:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward>)

lr = 0.000010
epoch 0 train_loss: 0.221


[[4]]
[[4], [1]]
[[4], [1], [4]]
[[4], [1], [4], [7]]
[[4], [1], [4], [7], [6]]
[[4], [1], [4], [7], [6], [14]]
[[4], [1], [4], [7], [6], [14], [1]]
[[4], [1], [4], [7], [6], [14], [1], [4]]
[[4], [1], [4], [7], [6], [14], [1], [4], [4]]
[[4], [1], [4], [7], [6], [14], [1], [4], [4], [20]]
[[4], [1], [4], [7], [6], [14], [1], [4], [4], [20], [13]]
[[4], [1], [4], [7], [6], [14], [1], [4], [4], [20], [13], [9]]
[[4], [1], [4], [7], [6], [14], [1], [4], [4], [20], [13], [9], [16]]
[[4], [1], [4], [7], [6], [14], [1], [4], [4], [20], [13], [9], [16], [9]]
[[4], [1], [4], [7], [6], [14], [1], [4], [4], [20], [13], [9], [16], [9], [7]]
[[4], [1], [4], [7], [6], [14], [1], [4], [4], [20], [13], [9], [16], [9], [7], [10]]
[[4], [1], [4], [7], [6], [14], [1], [4], [4], [20], [13], [9], [16], [9], [7], [10], [10]]
[[4], [1], [4], [7], [6], [14], [1], [4], [4], [20], [13], [9], [16], [9], [7], [10], [10], [6]]
[[4], [1], [4], [7], [6], [14], [1], [4], [4], [20], [13], [9], [16], [9], [7], [10], 

tensor(0.0807, device='cuda:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward>)

lr = 0.000009
epoch 1 train_loss: 0.229


[[15]]
[[15], [4]]
[[15], [4], [0]]
[[15], [4], [0], [11]]
[[15], [4], [0], [11], [7]]
[[15], [4], [0], [11], [7], [9]]
[[15], [4], [0], [11], [7], [9], [4]]
[[15], [4], [0], [11], [7], [9], [4], [0]]
[[15], [4], [0], [11], [7], [9], [4], [0], [0]]
[[15], [4], [0], [11], [7], [9], [4], [0], [0], [20]]
[[15], [4], [0], [11], [7], [9], [4], [0], [0], [20], [13]]
[[15], [4], [0], [11], [7], [9], [4], [0], [0], [20], [13], [1]]
[[15], [4], [0], [11], [7], [9], [4], [0], [0], [20], [13], [1], [18]]
[[15], [4], [0], [11], [7], [9], [4], [0], [0], [20], [13], [1], [18], [1]]
[[15], [4], [0], [11], [7], [9], [4], [0], [0], [20], [13], [1], [18], [1], [1]]
[[15], [4], [0], [11], [7], [9], [4], [0], [0], [20], [13], [1], [18], [1], [1], [3]]
[[15], [4], [0], [11], [7], [9], [4], [0], [0], [20], [13], [1], [18], [1], [1], [3], [7]]
[[15], [4], [0], [11], [7], [9], [4], [0], [0], [20], [13], [1], [18], [1], [1], [3], [7], [2]]
[[15], [4], [0], [11], [7], [9], [4], [0], [0], [20], [13], [1], [18], 

tensor(0.4247, device='cuda:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward>)


KeyboardInterrupt: 

In [ ]:
print("hi")

In [28]:
sp_golds = validation_data['QUESTIONS'].apply(lambda x: x[0]['SHINT_']).tolist()
atype_golds = validation_data['QUESTIONS'].apply(lambda x: x[0]['ATYPE_']).tolist()

    
eval(dev_batches, 0, sp_golds, atype_golds)

[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35]]
[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35]]
[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35]]
[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 2

KeyboardInterrupt: 

SyntaxError: invalid syntax (<ipython-input-28-db5b9d01f4dc>, line 1)

In [48]:
#validation_data['QUESTIONS'].apply(lambda x: x[0]['SHINT_']).tolist()

In [43]:
test_batch = iter(dataloader_dev).next()
with torch.no_grad():
    print(network.predict_fgc(test_batch))

{'sp': [3], 'sp_scores': [0.053327079862356186, 0.05332055315375328, 0.05332907661795616, 0.05333000421524048]}


In [49]:
len(dataloader_train)

7856

In [50]:
len(dataloader_dev)

2211

In [60]:
validation_data.shape

(247, 5)